In [9]:
import pandas as pd
from datetime import datetime
import os
import math

def load_stock_data(file_path):
    """Load and prepare stock data from CSV file"""
    try:
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date')
        return df
    except Exception as e:
        print(f"Error loading file {file_path}: {str(e)}")
        raise

def create_portfolio(output_path):
    # File paths
    file_paths = {
        'Eastern Housing': r"D:\Download Folder\Singer Bangladesh Stock Price History.csv",
        'BATBC': r"D:\Download Folder\British American Tobacco Bangladesh Stock Price History.csv",
        'Beximco Pharma': r"D:\Download Folder\Beximco Pharmaceuticals Stock Price History.csv",
        'ACI Formulations': r"D:\Download Folder\ACI Formulations Stock Price History.csv",
        'Brac Bank': r"D:\Download Folder\Eastern Bank Stock Price History.csv"
    }
    
    # Target investment amount (we'll get close to this but not exactly)
    target_investment = 500000
    target_per_stock = target_investment / len(file_paths)
    
    # Define date range
    start_date = '2024-09-23'
    end_date = '2025-01-16'
    
    # Load all stock data
    stock_data = {}
    for stock_name, file_path in file_paths.items():
        stock_data[stock_name] = load_stock_data(file_path)
    
    # Calculate whole number shares and actual investment for each stock
    initial_shares = {}
    actual_investments = {}
    for stock_name, df in stock_data.items():
        initial_price = df[df['Date'] >= start_date]['Price'].iloc[0]
        # Calculate whole number of shares using floor division
        shares = math.floor(target_per_stock / initial_price)
        initial_shares[stock_name] = shares
        # Calculate actual investment for this stock
        actual_investments[stock_name] = shares * initial_price
    
    # Calculate total actual investment
    total_actual_investment = sum(actual_investments.values())
    
    # Print investment summary
    print("\nInvestment Summary:")
    print(f"Target investment: {target_investment:,.2f} TK")
    print(f"Actual investment: {total_actual_investment:,.2f} TK")
    print(f"Difference: {target_investment - total_actual_investment:,.2f} TK")
    print("\nPer Stock Breakdown:")
    for stock_name in file_paths.keys():
        print(f"\n{stock_name}:")
        print(f"Shares purchased: {initial_shares[stock_name]:,}")
        print(f"Investment amount: {actual_investments[stock_name]:,.2f} TK")
    
    # Initialize results DataFrame
    date_range = pd.date_range(start=start_date, end=end_date)
    portfolio_df = pd.DataFrame(index=date_range)
    portfolio_df.index.name = 'Date'
    
    # Track daily portfolio values
    for date in date_range:
        daily_value = 0
        
        for stock_name, df in stock_data.items():
            price_data = df[df['Date'] <= date].iloc[-1]
            current_price = price_data['Price']
            shares = initial_shares[stock_name]
            
            stock_value = shares * current_price
            daily_value += stock_value
            
            portfolio_df.loc[date, f'{stock_name}_Price'] = current_price
            portfolio_df.loc[date, f'{stock_name}_Value'] = stock_value
            portfolio_df.loc[date, f'{stock_name}_Daily_Change'] = price_data['Change %']
            
        portfolio_df.loc[date, 'Total_Portfolio_Value'] = daily_value
        portfolio_df.loc[date, 'Return_Amount'] = daily_value - total_actual_investment
        portfolio_df.loc[date, 'Return_Percentage'] = ((daily_value - total_actual_investment) / total_actual_investment) * 100
    
    portfolio_df['Initial_Investment'] = total_actual_investment
    
    for stock_name, shares in initial_shares.items():
        portfolio_df[f'{stock_name}_Shares'] = shares
        portfolio_df[f'{stock_name}_Initial_Investment'] = actual_investments[stock_name]
    
    return portfolio_df

# Run the portfolio tracking
if __name__ == "__main__":
    try:
        # Specify output directory
        output_dir = r"D:\Download Folder\\"
        
        # Create output filename with timestamp
        
        output_filename = f"subashis_portfolio_tracking.csv"
        output_path = os.path.join(output_dir, output_filename)
        
        # Create portfolio DataFrame
        portfolio_df = create_portfolio(output_path)
        
        # Save to CSV with error handling
        try:
            portfolio_df.to_csv(output_path)
            print(f"\nPortfolio tracking data has been saved to {output_path}")
        except PermissionError:
            alternative_path = os.path.join(os.path.expanduser("~"), "Documents", output_filename)
            print(f"Could not save to {output_path}. Trying alternative location: {alternative_path}")
            portfolio_df.to_csv(alternative_path)
            print(f"Portfolio tracking data has been saved to {alternative_path}")
        
        # Display summary of the latest day
        latest_day = portfolio_df.iloc[-1]
        print("\nLatest Portfolio Summary:")
        print("Date:", latest_day.name.date())
        print("Total Portfolio Value: {:.2f} TK".format(latest_day['Total_Portfolio_Value']))
        print("Total Return: {:.2f} TK ({:.2f}%)".format(
            latest_day['Return_Amount'],
            latest_day['Return_Percentage']
        ))
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")


Investment Summary:
Target investment: 500,000.00 TK
Actual investment: 499,755.30 TK
Difference: 244.70 TK

Per Stock Breakdown:

Eastern Housing:
Shares purchased: 717
Investment amount: 99,949.80 TK

BATBC:
Shares purchased: 254
Investment amount: 99,974.40 TK

Beximco Pharma:
Shares purchased: 1,345
Investment amount: 99,933.50 TK

ACI Formulations:
Shares purchased: 725
Investment amount: 99,905.00 TK

Brac Bank:
Shares purchased: 3,802
Investment amount: 99,992.60 TK

Portfolio tracking data has been saved to D:\Download Folder\\subashis_portfolio_tracking.csv

Latest Portfolio Summary:
Date: 2025-01-16
Total Portfolio Value: 443807.60 TK
Total Return: -55947.70 TK (-11.20%)
